In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import pickle
import gsdmm
from gsdmm import MovieGroupProcess
from tqdm import tqdm

Ler o CSV criado previamente e converter tokens em listas de tokens

In [10]:
caminho_arquivo = r'C:\Users\Pessoal\Documents\Unifesp\Aries\aries_topic_selector\Noise_remover\tweets_processados.csv'
tweets_df = pd.read_csv(caminho_arquivo)
tweets_df.dropna(axis='columns', inplace=True)
tweets_df['tokens'] = tweets_df.tokens.apply(lambda x:  re.split('\s', x))
docs = tweets_df['tokens'].tolist()

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Pessoal\AppData\Local\Temp\ipykernel_12864\783247071.py:4: SyntaxWarning: invalid escape sequence '\s'
  tweets_df['tokens'] = tweets_df.tokens.apply(lambda x:  re.split('\s', x))


# Treinamento do Modelo usando o GSDMM

Agora, aplicaremos o GSDMM (Gibbs Sampling Dirichlet Mixture Model) , com os seguintes parâmetros:
- K — É o número máximo de topicos a serem encontrados
- Alpha α - Determina o quão fácil mesas pequenas são removidas
- Beta β - Controla se uma mesa é escolhida por popularidade ou similaridade. Quanto menor, mais popular
- N_iters - Número de vezes em que um Subject é realocado numa nova mesa

In [11]:
mgp = MovieGroupProcess(K=10, alpha=0.1, beta=0.1, n_iters=30)
vocab = set(x for doc in docs for x in doc)
n_terms = len(vocab)
y = mgp.fit(docs, n_terms)


with open("C:\\Users\\Pessoal\\Documents\\Unifesp\Aries\\aries_topic_selector\\modelos_treinados\\10clusters.model", 'wb') as f:
    pickle.dump(mgp, f)
    f.close()

<>:7: SyntaxWarning: invalid escape sequence '\A'
<>:7: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Pessoal\AppData\Local\Temp\ipykernel_12864\3350084293.py:7: SyntaxWarning: invalid escape sequence '\A'
  with open("C:\\Users\\Pessoal\\Documents\\Unifesp\Aries\\aries_topic_selector\\modelos_treinados\\10clusters.model", 'wb') as f:


In stage 0: transferred 16020 clusters with 10 clusters populated
In stage 1: transferred 10844 clusters with 10 clusters populated
In stage 2: transferred 6986 clusters with 10 clusters populated
In stage 3: transferred 4953 clusters with 10 clusters populated
In stage 4: transferred 3878 clusters with 10 clusters populated
In stage 5: transferred 3401 clusters with 10 clusters populated
In stage 6: transferred 2853 clusters with 10 clusters populated
In stage 7: transferred 2565 clusters with 10 clusters populated
In stage 8: transferred 2384 clusters with 10 clusters populated
In stage 9: transferred 2275 clusters with 10 clusters populated
In stage 10: transferred 2137 clusters with 10 clusters populated
In stage 11: transferred 2093 clusters with 10 clusters populated
In stage 12: transferred 2067 clusters with 10 clusters populated
In stage 13: transferred 1975 clusters with 10 clusters populated
In stage 14: transferred 2037 clusters with 10 clusters populated
In stage 15: trans

# Explorando os Tópicos

Definiremos algumas funções para ajudar a explorar os tópicos

In [12]:
def top_words(cluster_word_distribution, top_cluster, values):
    '''prints the top words in each cluster'''
    for cluster in top_cluster:
        sort_dicts =sorted(mgp.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print('Cluster %s : %s'%(cluster,sort_dicts))
        print(' — — — — — — — — —')

def cluster_importance(mgp):
    '''returns a word-topic matrix[phi] where each value represents
    the word importance for that particular cluster;
    phi[i][w] would be the importance of word w in topic i.
    '''
    n_z_w = mgp.cluster_word_distribution
    beta, V, K = mgp.beta, mgp.vocab_size, mgp.K
    phi = [{} for i in range(K)]
    for z in range(K):
        for w in n_z_w[z]:
            phi[z][w] = (n_z_w[z][w]+beta)/(sum(n_z_w[z].values())+V*beta)
    return phi

def topic_allocation(df, docs, mgp, topic_dict):
    '''allocates all topics to each document in original dataframe,
    adding two columns for cluster number and cluster description'''
    topic_allocations = []
    for doc in tqdm(docs):
        topic_label, score = mgp.choose_best_label(doc)
        topic_allocations.append(topic_label)

    df['cluster'] = topic_allocations

    df['topic_name'] = df.cluster.apply(lambda x: get_topic_name(x, topic_dict))
    print('Complete. Number of documents with topic allocated: {}'.format(len(df)))

def get_topic_name(doc, topic_dict):
    '''returns the topic name string value from a dictionary of topics'''
    topic_desc = topic_dict[doc]
    return topic_desc

Insight estatístico

In [13]:
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)
print('*'*20)
# topics sorted by the number of documents they are allocated to
top_index = doc_count.argsort()[-10:][::-1]
print('Most important clusters (by number of docs inside):',top_index)
print('*'*20)
# show the top 5 words in term frequency for each cluster 
topic_indices = np.arange(start=0, stop=len(doc_count), step=1)
top_words(mgp.cluster_word_distribution, topic_indices, 5)

Number of documents per topic : [ 106 1211 2728  938 4068 1238  246 1704 7105  227]
********************
Most important clusters (by number of docs inside): [8 4 2 7 5 1 3 6 9 0]
********************
Cluster 0 : [('amoxicilino', 27), ('tuberculose', 22), ('água', 14), ('amoxicilina', 12), ('remédio', 10)]
 — — — — — — — — —
Cluster 1 : [('infecção', 789), ('urinário', 601), ('tuberculose', 366), ('xixi', 155), ('doença', 145)]
 — — — — — — — — —
Cluster 2 : [('bactéria', 2126), ('antibiótico', 1702), ('resistência', 1190), ('resistente', 1006), ('tomar', 488)]
 — — — — — — — — —
Cluster 3 : [('antibiotico', 505), ('che', 336), ('non', 334), ('tuberculose', 317), ('per', 262)]
 — — — — — — — — —
Cluster 4 : [('antibiótico', 1834), ('infecção', 1282), ('urinário', 859), ('médico', 829), ('tomar', 620)]
 — — — — — — — — —
Cluster 5 : [('tuberculose', 1224), ('doença', 360), ('saúde', 240), ('tratamento', 224), ('caso', 173)]
 — — — — — — — — —
Cluster 6 : [('una', 56), ('atendimento', 54)

Se quisermos ver a importância de algumas palavras , usando a função cluster_importance

Agora, decidimos o que é  cada tópico

In [14]:
topic_dict = {}
topic_names = ['Cluster 0,',
                'Cluster 1,',
                'Cluster 2,',
                'Cluster 3,',
                'Cluster 4,',
                'Cluster 5,',
                'Cluster 6,',
                'Cluster 7,',
                'Cluster 8,',
                'Cluster 9,']

for i, topic_num in enumerate(topic_indices):
 topic_dict[topic_num]=topic_names[i]



topic_allocation(tweets_df, docs, mgp, topic_dict)

100%|██████████| 19571/19571 [00:06<00:00, 3124.01it/s]


Complete. Number of documents with topic allocated: 19571


In [15]:
tweets_df.to_excel('sttm_10topics_results.xlsx', index = False,  header=True)

# Aplicação do Método para Notícias do G1

In [7]:
caminho_arquivo = r'C:\Users\Pessoal\Documents\Unifesp\Aries\aries_topic_selector\Noise_remover\noticias_processados.csv'
news_df = pd.read_csv(caminho_arquivo)
news_df.dropna(axis='columns', inplace=True)
news_df['tokens'] = news_df.tokens.apply(lambda x:  re.split('\s', x))
docs = news_df['tokens'].tolist()

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Pessoal\AppData\Local\Temp\ipykernel_12864\3873121176.py:4: SyntaxWarning: invalid escape sequence '\s'
  news_df['tokens'] = news_df.tokens.apply(lambda x:  re.split('\s', x))


# Treinamento do Modelo usando o GSDMM

In [8]:
mgp2 = MovieGroupProcess(K=10, alpha=0.1, beta=0.1, n_iters=30)
vocab = set(x for doc in docs for x in doc)
n_terms = len(vocab)
y = mgp2.fit(docs, n_terms)


with open("C:\\Users\\Pessoal\\Documents\\Unifesp\Aries\\aries_topic_selector\\modelos_treinados\\10clusters.model", 'wb') as f:
    pickle.dump(mgp2, f)
    f.close()

<>:7: SyntaxWarning: invalid escape sequence '\A'
<>:7: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Pessoal\AppData\Local\Temp\ipykernel_12864\2481954221.py:7: SyntaxWarning: invalid escape sequence '\A'
  with open("C:\\Users\\Pessoal\\Documents\\Unifesp\Aries\\aries_topic_selector\\modelos_treinados\\10clusters.model", 'wb') as f:


In stage 0: transferred 168 clusters with 9 clusters populated
In stage 1: transferred 23 clusters with 6 clusters populated
In stage 2: transferred 6 clusters with 6 clusters populated
In stage 3: transferred 1 clusters with 6 clusters populated
In stage 4: transferred 0 clusters with 6 clusters populated
In stage 5: transferred 2 clusters with 6 clusters populated
In stage 6: transferred 1 clusters with 6 clusters populated
In stage 7: transferred 0 clusters with 6 clusters populated
In stage 8: transferred 1 clusters with 6 clusters populated
In stage 9: transferred 0 clusters with 6 clusters populated
In stage 10: transferred 1 clusters with 6 clusters populated
In stage 11: transferred 1 clusters with 6 clusters populated
In stage 12: transferred 0 clusters with 6 clusters populated
In stage 13: transferred 2 clusters with 6 clusters populated
In stage 14: transferred 1 clusters with 6 clusters populated
In stage 15: transferred 1 clusters with 6 clusters populated
In stage 16: tr

Insight estatístico

In [9]:
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)
print('*'*20)
# topics sorted by the number of documents they are allocated to
top_index = doc_count.argsort()[-10:][::-1]
print('Most important clusters (by number of docs inside):',top_index)
print('*'*20)
# show the top 5 words in term frequency for each cluster 
topic_indices = np.arange(start=0, stop=len(doc_count), step=1)
top_words(mgp.cluster_word_distribution, topic_indices, 5)

Number of documents per topic : [  0   0   0  17  13   0   4   0  11 139]
********************
Most important clusters (by number of docs inside): [9 3 4 8 6 7 5 2 1 0]
********************
Cluster 0 : []
 — — — — — — — — —
Cluster 1 : []
 — — — — — — — — —
Cluster 2 : []
 — — — — — — — — —
Cluster 3 : [('magaziner', 42), ('luizar', 33), ('hospital', 32), ('fnac', 30), ('ano', 21)]
 — — — — — — — — —
Cluster 4 : [('magaziner', 34), ('luizar', 28), ('hospital', 27), ('fnac', 26), ('chico', 23)]
 — — — — — — — — —
Cluster 5 : []
 — — — — — — — — —
Cluster 6 : [('magaziner', 9), ('luizar', 7), ('rio', 7), ('feira', 7), ('ano', 6)]
 — — — — — — — — —
Cluster 7 : []
 — — — — — — — — —
Cluster 8 : [('magaziner', 30), ('luizar', 24), ('fnac', 22), ('pneumonia', 19), ('feira', 15)]
 — — — — — — — — —
Cluster 9 : [('pneumonia', 328), ('ano', 280), ('hospital', 253), ('magaziner', 244), ('médico', 233)]
 — — — — — — — — —


In [ ]:
topic_dict = {}
topic_names = ['Cluster 0,',
                'Cluster 1,',
                'Cluster 2,',
                'Cluster 3,',
                'Cluster 4,',
                'Cluster 5,',
                'Cluster 6,',
                'Cluster 7,',
                'Cluster 8,',
                'Cluster 9,']

for i, topic_num in enumerate(topic_indices):
 topic_dict[topic_num]=topic_names[i]



topic_allocation(tweets_df, docs, mgp, topic_dict)

100%|██████████| 4268/4268 [00:01<00:00, 3300.27it/s]

Complete. Number of documents with topic allocated: 4268


In [ ]:
tweets_df.to_excel('sttm_10topics_results.xlsx', index = False,  header=True)